# Crucible Project and User Management

This notebook demonstrates how to manage projects and users in Crucible using the Python client.

**Note:** Most operations in this notebook require **admin permissions**.

**Prerequisites:**
- Admin API credentials configured (run `crucible config init` in terminal)
- Admin access to Crucible

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Creating a Project](#create-project)
3. [Getting Project Details](#get-project)
4. [Listing Projects](#list-projects)
5. [Creating Users](#create-user)
6. [Getting User Details](#get-user)
7. [Adding Users to Projects](#add-user-to-project)
8. [Getting Project Users](#get-project-users)

<a id='setup'></a>
## 1. Setup and Configuration

First, configure your Crucible credentials. Run this command in your terminal:

```bash
crucible config init
```

Make sure to use an **admin API key** for the operations in this notebook.

**Alternative: Without Terminal Access**

If you don't have terminal access (e.g., JupyterHub, Google Colab, VSCode Flatpak), you can initialize the client directly with your admin credentials:

```python
client = CrucibleClient(
    api_url="https://crucible.lbl.gov/api/v1",
    api_key="your-admin-api-key-here"
)
```

Then import and initialize the client:

In [ ]:
from crucible.client import CrucibleClient

# Initialize the client (automatically loads configuration)
client = CrucibleClient()

print("✓ Crucible client initialized successfully!")

<a id='create-project'></a>
## 2. Creating a Project

Create a new project or get an existing one using `get_or_create()`.

**Required fields:**
- `project_id`: Unique identifier for the project (e.g., "my-new-project")
- `organization`: Organization name (e.g., "Molecular Foundry", "LBNL")
- `project_lead_email`: Email of the project lead

In [ ]:
# Create or get a project
project = client.projects.get_or_create(
    project_id="my-demo-project",
    organization="Molecular Foundry",
    project_lead_email="your.email@lbl.gov"
)

print(f"✓ Project: {project['project_id']}")
print(f"  Organization: {project.get('organization', 'N/A')}")
print(f"  Lead: {project.get('project_lead_email', 'N/A')}")
print(f"  Created: {project.get('creation_time', 'N/A')[:10] if project.get('creation_time') else 'N/A'}")

<a id='get-project'></a>
## 3. Getting Project Details

Retrieve details of an existing project by its ID:

In [ ]:
# Get project details
project_id = "crucible-demo"  # Example project ID
project_details = client.projects.get(project_id)

print(f"Project: {project_details['project_id']}")
print(f"Organization: {project_details.get('organization', 'N/A')}")
print(f"Project Lead: {project_details.get('project_lead_email', 'N/A')}")
print(f"Description: {project_details.get('description', 'No description')}")
print(f"Created: {project_details.get('creation_time', 'N/A')}")

<a id='list-projects'></a>
## 4. Listing Projects

List all accessible projects or filter by user ORCID:

In [ ]:
# List all accessible projects
projects = client.projects.list(limit=10)

print(f"Found {len(projects)} accessible project(s)\n")

for i, proj in enumerate(projects, 1):
    print(f"{i}. {proj['project_id']}")
    print(f"   Organization: {proj.get('organization', 'N/A')}")
    print(f"   Lead: {proj.get('project_lead_email', 'N/A')}")
    print()

You can also filter projects by user ORCID to see which projects a specific user has access to:

In [ ]:
# List projects for a specific user
user_orcid = "0000-0001-2345-6789"  # Replace with actual ORCID

user_projects = client.projects.list(orcid=user_orcid)

print(f"Projects accessible to {user_orcid}:")
for proj in user_projects:
    print(f"  - {proj['project_id']}: {proj.get('organization', 'N/A')}")

<a id='create-user'></a>
## 5. Creating Users

Add a new user to the Crucible system.

**Required fields:**
- `first_name`: User's first name
- `last_name`: User's last name
- `orcid`: User's ORCID identifier (format: 0000-0000-0000-000X)
- `projects`: List of project IDs the user should have access to

**Optional fields:**
- `email`: Primary email address
- `lbl_email`: LBL email address (if applicable)

In [ ]:
# Create a new user
user_info = {
    "first_name": "Jane",
    "last_name": "Doe",
    "orcid": "0000-0001-2345-6789",
    "email": "jane.doe@example.com",
    "lbl_email": "jdoe@lbl.gov",  # Optional
    "projects": ["my-demo-project", "crucible-demo"]  # Projects to grant access to
}

new_user = client.users.create(user_info)

print(f"✓ User created successfully!")
print(f"  Name: {new_user.get('first_name')} {new_user.get('last_name')}")
print(f"  ORCID: {new_user.get('orcid')}")
print(f"  Email: {new_user.get('email', 'N/A')}")
print(f"  Projects: {len(user_info['projects'])}")

<a id='get-user'></a>
## 6. Getting User Details

Retrieve user information by ORCID or email:

### Get User by ORCID

In [ ]:
# Get user by ORCID
orcid = "0000-0001-2345-6789"
user = client.users.get(orcid=orcid)

if user:
    print(f"User found: {user.get('first_name')} {user.get('last_name')}")
    print(f"  ORCID: {user.get('orcid')}")
    print(f"  Email: {user.get('email', 'N/A')}")
    print(f"  LBL Email: {user.get('lbl_email', 'N/A')}")
    print(f"  Created: {user.get('creation_time', 'N/A')}")
else:
    print(f"User with ORCID {orcid} not found")

### Get User by Email

In [ ]:
# Get user by email
email = "jane.doe@example.com"
user = client.users.get(email=email)

if user:
    print(f"User found: {user.get('first_name')} {user.get('last_name')}")
    print(f"  ORCID: {user.get('orcid')}")
    print(f"  Email: {user.get('email', 'N/A')}")
else:
    print(f"User with email {email} not found")

<a id='add-user-to-project'></a>
## 7. Adding Users to Projects

Grant an existing user access to a project:

In [ ]:
# Add user to project
user_orcid = "0000-0001-2345-6789"
project_id = "my-demo-project"

result = client.projects.add_user(orcid=user_orcid, project_id=project_id)

print(f"✓ User {user_orcid} added to project {project_id}")
print(f"\nProject now has {len(result)} user(s) (excluding project lead)")

<a id='get-project-users'></a>
## 8. Getting Project Users

List all users who have access to a specific project:

**Note:** This returns project team members but excludes the project lead.

In [ ]:
# Get project users
project_id = "crucible-demo"
users = client.projects.get_users(project_id)

print(f"Project {project_id} has {len(users)} team member(s) (excluding lead)\n")

for i, user in enumerate(users, 1):
    print(f"{i}. {user.get('first_name', '')} {user.get('last_name', '')}")
    print(f"   ORCID: {user.get('orcid', 'N/A')}")
    print(f"   Email: {user.get('email', 'N/A')}")
    print()

## Summary

This notebook demonstrated the core project and user management operations:

### Projects
- ✓ **Create/Get Project** - `client.projects.get_or_create()`
- ✓ **Get Project Details** - `client.projects.get(project_id)`
- ✓ **List Projects** - `client.projects.list()` or `client.projects.list(orcid=...)`
- ✓ **Get Project Users** - `client.projects.get_users(project_id)`
- ✓ **Add User to Project** - `client.projects.add_user(orcid, project_id)`

### Users
- ✓ **Create User** - `client.users.create(user_info)`
- ✓ **Get User by ORCID** - `client.users.get(orcid=...)`
- ✓ **Get User by Email** - `client.users.get(email=...)`

### Additional Resources

- **Main Tutorial** - See `crucible_tutorial.ipynb` for dataset and sample operations
- **CLI Usage** - Run `crucible --help` to see command-line options
- **API Reference** - Check docstrings in `crucible/resources/` for detailed method documentation

## CLI Equivalents

Most operations in this notebook can also be performed via the command line:

```bash
# Create a project
crucible project create --project-id my-project -o "Molecular Foundry" -e lead@lbl.gov

# Get project details
crucible project get my-project

# List projects
crucible project list

# Create a user
crucible user create

# Get user details
crucible user get 0000-0001-2345-6789

# Add user to project
crucible project add-user my-project 0000-0001-2345-6789

# Get project users
crucible project get-users my-project

# Open project in browser
crucible open my-project
```